In [1]:
import org.apache.spark.sql.functions.{broadcast, split, lit, col}
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://ac902101fe0e:4045
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1696193087596)
SparkSession available as 'spark'


import org.apache.spark.sql.functions.{broadcast, split, lit, col}
import org.apache.spark.sql.SparkSession


In [2]:
// Starting the new Spark session
val sparkSession = SparkSession.builder.appName("H6SparkSession").getOrCreate()

sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3b4510dd


In [3]:
sparkSession.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [14]:
// Match Details
val matchDetails = sparkSession.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/home/iceberg/data/match_details.csv")

// Matches
val matches = sparkSession.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/home/iceberg/data/matches.csv")

// Medals Matches Players
val medalsMatchesPlayers = sparkSession.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/home/iceberg/data/medals_matches_players.csv")

// Medals
val medals = sparkSession.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/home/iceberg/data/medals.csv")

// Maps
val maps = sparkSession.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/home/iceberg/data/maps.csv")

matchDetails: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]
matches: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
medalsMatchesPlayers: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 2 more fields]
medals: org.apache.spark.sql.DataFrame = [medal_id: bigint, sprite_uri: string ... 10 more fields]
maps: org.apache.spark.sql.DataFrame = [mapid: string, name: string ... 1 more field]


In [15]:
// Medals dataset
val matchesExpanded = matches.join(
    broadcast(maps), Seq("mapid"), "left")
    .select(
        $"match_id",
        $"is_team_game",
        $"playlist_id",
        $"game_variant_id",
        $"is_match_over",
        $"completion_date",
        $"match_duration",
        $"game_mode",
        $"map_variant_id",
        $"mapid",
        $"name" as "map_name",
        $"description" as "map_description",
    )
// Maps dataset
val medalsMatchesPlayersExpanded = medalsMatchesPlayers.join(
    broadcast(medals), Seq("medal_id"), "left")
    .select(
        $"match_id",
        $"player_gamertag",
        $"count",
        $"medal_id",
        $"sprite_uri",
        $"sprite_left",
        $"sprite_top",
        $"sprite_sheet_width",
        $"sprite_sheet_height",
        $"sprite_width",
        $"sprite_height",
        $"classification",
        $"description" as "medal_description",
        $"name" as "medal_name",
        $"difficulty",
    )

matchesExpanded: org.apache.spark.sql.DataFrame = [match_id: string, is_team_game: boolean ... 10 more fields]
medalsMatchesPlayersExpanded: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 13 more fields]


In [17]:
// Dropping table if necessary
spark.sql("DROP TABLE IF EXISTS bootcamp.match_details_in_buckets;")

// Defining the table for 'match_details'
val matchesDetailsBuckets = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_in_buckets (
        match_id STRING
    ,   player_gamertag STRING
    ,   previous_spartan_rank INTEGER
    ,   spartan_rank INTEGER
    ,   previous_total_xp INTEGER
    ,   total_xp INTEGER
    ,   previous_csr_tier INTEGER
    ,   previous_csr_designation INTEGER
    ,   previous_csr INTEGER
    ,   previous_csr_percent_to_next_tier INTEGER
    ,   previous_csr_rank INTEGER
    ,   current_csr_tier INTEGER
    ,   current_csr_designation INTEGER
    ,   current_csr INTEGER
    ,   current_csr_percent_to_next_tier INTEGER
    ,   current_csr_rank INTEGER
    ,   player_rank_on_team INTEGER
    ,   player_finished BOOLEAN
    ,   player_average_life STRING
    ,   player_total_kills INTEGER
    ,   player_total_headshots INTEGER
    ,   player_total_weapon_damage DOUBLE
    ,   player_total_shots_landed INTEGER
    ,   player_total_melee_kills INTEGER
    ,   player_total_melee_damage DOUBLE
    ,   player_total_assassinations INTEGER
    ,   player_total_ground_pound_kills INTEGER
    ,   player_total_shoulder_bash_kills INTEGER
    ,   player_total_grenade_damage DOUBLE
    ,   player_total_power_weapon_damage DOUBLE
    ,   player_total_power_weapon_grabs INTEGER
    ,   player_total_deaths INTEGER
    ,   player_total_assists INTEGER
    ,   player_total_grenade_kills INTEGER
    ,   did_win INTEGER
    ,   team_id INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""

// Running SQL command
spark.sql(matchesDetailsBuckets)

// Saving results to table
matchDetails
    .write
    .mode("append")
    .bucketBy(16, "match_id")
    .saveAsTable("bootcamp.match_details_in_buckets")

matchesDetailsBuckets: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.match_details_in_buckets (
        match_id STRING
    ,   player_gamertag STRING
    ,   previous_spartan_rank INTEGER
    ,   spartan_rank INTEGER
    ,   previous_total_xp INTEGER
    ,   total_xp INTEGER
    ,   previous_csr_tier INTEGER
    ,   previous_csr_designation INTEGER
    ,   previous_csr INTEGER
    ,   previous_csr_percent_to_next_tier INTEGER
    ,   previous_csr_rank INTEGER
    ,   current_csr_tier INTEGER
    ,   current_csr_designation INTEGER
    ,   current_csr INTEGER
    ,   current_csr_percent_to_next_tier INTEGER
    ,   current_csr_rank INTEGER
    ,   player_rank_on_team INTEGER
    ,   player_finished BOOLEAN
    ,   player_average_life STRING
    ,   player_total_kills INTEGER
    ,   pl...


In [18]:
// Dropping table if necessary
spark.sql("DROP TABLE IF EXISTS bootcamp.matches_expanded_in_buckets;")

// Defining the table for 'match_details'
val matchesExpandedBuckets = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_expanded_in_buckets (
        match_id STRING
    ,   is_team_game BOOLEAN
    ,   playlist_id STRING
    ,   game_variant_id STRING
    ,   is_match_over BOOLEAN
    ,   completion_date TIMESTAMP
    ,   match_duration STRING
    ,   game_mode STRING
    ,   map_variant_id STRING
    ,   mapid STRING
    ,   map_name STRING
    ,   map_description STRING
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""

// Running SQL command
spark.sql(matchesExpandedBuckets)

// Saving results to table
matchesExpanded
    .write
    .mode("append")
    .bucketBy(16, "match_id")
    .saveAsTable("bootcamp.matches_expanded_in_buckets")

matchesExpandedBuckets: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.matches_expanded_in_buckets (
        match_id STRING
    ,   is_team_game BOOLEAN
    ,   playlist_id STRING
    ,   game_variant_id STRING
    ,   is_match_over BOOLEAN
    ,   completion_date TIMESTAMP
    ,   match_duration STRING
    ,   game_mode STRING
    ,   map_variant_id STRING
    ,   mapid STRING
    ,   map_name STRING
    ,   map_description STRING
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"


In [19]:
// Dropping table if necessary
spark.sql("DROP TABLE IF EXISTS bootcamp.medal_matches_players_expanded_in_buckets;")

// Defining the table for 'match_details'
val medalMatchesPlayersBuckets = """
CREATE TABLE IF NOT EXISTS bootcamp.medal_matches_players_expanded_in_buckets (
        match_id STRING
    ,   player_gamertag STRING
    ,   count INTEGER
    ,   medal_id LONG
    ,   sprite_uri STRING
    ,   sprite_left INTEGER
    ,   sprite_top INTEGER
    ,   sprite_sheet_width INTEGER
    ,   sprite_sheet_height INTEGER
    ,   sprite_width INTEGER
    ,   sprite_height INTEGER
    ,   classification STRING
    ,   medal_description STRING
    ,   medal_name STRING
    ,   difficulty INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""

// Running SQL command
spark.sql(medalMatchesPlayersBuckets)

// Saving results to table
medalsMatchesPlayersExpanded
    .write
    .mode("append")
    .bucketBy(16, "match_id")
    .saveAsTable("bootcamp.medal_matches_players_expanded_in_buckets")

medalMatchesPlayersBuckets: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.medal_matches_players_expanded_in_buckets (
        match_id STRING
    ,   player_gamertag STRING
    ,   count INTEGER
    ,   medal_id LONG
    ,   sprite_uri STRING
    ,   sprite_left INTEGER
    ,   sprite_top INTEGER
    ,   sprite_sheet_width INTEGER
    ,   sprite_sheet_height INTEGER
    ,   sprite_width INTEGER
    ,   sprite_height INTEGER
    ,   classification STRING
    ,   medal_description STRING
    ,   medal_name STRING
    ,   difficulty INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"


In [20]:
val matchDetailsBucketTable = spark.table("bootcamp.match_details_in_buckets")

val matchesExpanedBucketTable = spark.table("bootcamp.matches_expanded_in_buckets")

val medalMatchesPlayersExpandedBucketTable = spark.table("bootcamp.medal_matches_players_expanded_in_buckets")

matchDetailsBucketTable: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]
matchesExpanedBucketTable: org.apache.spark.sql.DataFrame = [match_id: string, is_team_game: boolean ... 10 more fields]
medalMatchesPlayersExpandedBucketTable: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 13 more fields]


In [21]:
// Joining tables by 'match_id'
val combinedDataset = matchesExpanedBucketTable
        .join(matchDetailsBucketTable, Seq("match_id"))
        .join(medalMatchesPlayersExpandedBucketTable, Seq("match_id"), "left")

combinedDataset: org.apache.spark.sql.DataFrame = [match_id: string, is_team_game: boolean ... 59 more fields]


In [22]:
val aggregatedDatasetPlayers = combinedDataset
        .groupBy(
                "match_id"
            ,   "playlist_id"
            ,   "mapid"
            ,   "map_name"
            ,   "demo.bootcamp.match_details_in_buckets.player_gamertag"
            ,   "classification"
        )
        .agg(max($"player_total_kills").as("player_most_average_kills"))


+--------------------+--------------------+--------------------+--------+---------------+-----------------+-------------------------+
|            match_id|         playlist_id|               mapid|map_name|player_gamertag|   classification|player_most_average_kills|
+--------------------+--------------------+--------------------+--------+---------------+-----------------+-------------------------+
|00169217-cca6-4b4...|2323b76a-db98-4e0...|cc040aa1-f206-11e...|  Fathom|  King Terror V|WeaponProficiency|                       14|
|00169217-cca6-4b4...|2323b76a-db98-4e0...|cc040aa1-f206-11e...|  Fathom|  King Terror V|            Style|                       14|
|00169217-cca6-4b4...|2323b76a-db98-4e0...|cc040aa1-f206-11e...|  Fathom|  King Terror V|        MultiKill|                       14|
|00169217-cca6-4b4...|2323b76a-db98-4e0...|cc040aa1-f206-11e...|  Fathom|  King Terror V|     KillingSpree|                       14|
|00169217-cca6-4b4...|2323b76a-db98-4e0...|cc040aa1-f206-11e..

aggregatedDatasetPlayers: org.apache.spark.sql.DataFrame = [match_id: string, playlist_id: string ... 5 more fields]


In [23]:
//Most kills per game
aggregatedDatasetPlayers
    .groupBy("player_gamertag")
    .agg(avg("player_most_average_kills").as("player_average_kills"))
    .orderBy(col("player_average_kills").desc)
    .limit(1)
    .select("player_gamertag", "player_average_kills")
    .show(false)

+---------------+--------------------+
|player_gamertag|player_average_kills|
+---------------+--------------------+
|gimpinator14   |109.0               |
+---------------+--------------------+



In [24]:
//Playlist gets played the most
aggregatedDatasetPlayers
    .groupBy("playlist_id")
    .agg(count("playlist_id").as("total_matches_played"))
    .orderBy(col("total_matches_played").desc)
    .limit(1)
    .select("playlist_id")
    .show(false)

+------------------------------------+
|playlist_id                         |
+------------------------------------+
|f72e0ef0-7c4a-4307-af78-8e38dac3fdba|
+------------------------------------+



In [25]:
//Map gets played the most?
aggregatedDatasetPlayers
    .groupBy("mapid", "map_name")
    .agg(count("mapid").as("total_played"))
    .orderBy(col("total_played").desc)
    .limit(1)
    .select("map_name", "mapid")
    .show(false)

+--------------+------------------------------------+
|map_name      |mapid                               |
+--------------+------------------------------------+
|Breakout Arena|c7edbf0f-f206-11e4-aa52-24be05e24f7e|
+--------------+------------------------------------+



In [26]:
//Which map do players get the most Killing Spree on?
aggregatedDatasetPlayers
    .filter(col("classification") === "KillingSpree")
    .groupBy("mapid", "map_name")
    .agg(count("mapid").as("total_counts"))
    .orderBy(col("total_counts").desc)
    .limit(1)
    .select("map_name", "mapid")
    .show(false)

+--------------+------------------------------------+
|map_name      |mapid                               |
+--------------+------------------------------------+
|Breakout Arena|c7edbf0f-f206-11e4-aa52-24be05e24f7e|
+--------------+------------------------------------+



In [34]:
//Get min number of partitions
aggregatedDatasetPlayers.rdd.getNumPartitions

res22: Int = 9


In [36]:
aggregatedDatasetPlayers
    .write
    .mode("overwrite")
    .saveAsTable("bootcamp.aggregatedDatasetUnsorted")

In [37]:
// Dropping table if necessary
spark.sql("DROP TABLE IF EXISTS bootcamp.sorted_by_playlist_id;")

// Defining the table object
val sortedByPlaylist = """
CREATE TABLE IF NOT EXISTS bootcamp.sorted_by_playlist_id (
        match_id STRING
    ,   playlist_id STRING
    ,   mapid STRING
    ,   map_name STRING
    ,   player_gamertag STRING
    ,   classification STRING
    ,   player_most_average_kills INTEGER
)
USING iceberg
;
"""

// Running SQL command
spark.sql(sortedByPlaylist)

sortedByPlaylist: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.sorted_by_playlist_id (
        match_id STRING
    ,   playlist_id STRING
    ,   mapid STRING
    ,   map_name STRING
    ,   player_gamertag STRING
    ,   classification STRING
    ,   player_most_average_kills INTEGER
)
USING iceberg
;
"
res25: org.apache.spark.sql.DataFrame = []


In [38]:
aggregatedDatasetPlayers
    .sortWithinPartitions("playlist_id")
    .write
    .mode("append")
    .saveAsTable("bootcamp.sorted_by_playlist_id")

In [39]:
spark.sql("""
SELECT
        SUM(file_size_in_bytes) as total_size
    ,   COUNT(1) AS number_of_files
    ,   'no sorting'
FROM bootcamp.aggregatedDatasetUnsorted.files
UNION ALL
SELECT
        SUM(file_size_in_bytes) as total_size
    ,   COUNT(1) AS number_of_files
    ,   'sorted'
FROM bootcamp.sorted_by_playlist_id.files
""").show(false)

+----------+---------------+----------+
|total_size|number_of_files|no sorting|
+----------+---------------+----------+
|2555557   |9              |no sorting|
|2478107   |9              |sorted    |
+----------+---------------+----------+

